In [12]:
def proceed(path: str) -> "np.ndarray":  
    import numpy as np 
    import pandas as pd 
    import os  
    import warnings
    warnings.filterwarnings('ignore') 
    import ast 
    from datetime import timedelta 
    import dill 
    import pickle 
    import xgboost as xgb
    with open("get_player_venue_stats.pkl", "rb") as f:
        get_player_venue_stats=dill.load(f) 
    with open("get_player_matchwise_stats.pkl", "rb") as f:
        get_player_matchwise_stats=dill.load(f)
    with open("get_player_vs_player_stats_ordered.pkl", "rb") as f:
        get_player_vs_player_stats_ordered=dill.load(f)
    with open("get_weather_data.pkl", "rb") as f:
        get_weather_data=dill.load(f)
    with open("get_fantasy_points.pkl", "rb") as f:
        get_fantasy_points=dill.load(f)  
    with open("xgb_model.pkl", "rb") as f:
        model=pickle.load(f) 
    player_vs_player_stats=pd.read_csv('dt/player_vs_player_stats.csv') 
    player_stats_with_date_venue=pd.read_csv('dt/odi_player_stats_with_date_venue.csv') 
    all_matches_player=pd.read_csv('dt/matches_all_players.csv') 
    fantasy_points=pd.read_csv('fantasy_points_data.csv') 
    venues_with_dates_with_locations_with_weather=pd.read_csv('dt/odis_venues_with_dates_with_locations_with_weather.csv')
    data=pd.read_csv(path)  
    def stack_and_pad_dataframes(df1, df2, df3, df4,x_train,counter):
        current_rows, num_columns = df1.shape
        required_rows=12
        if current_rows < required_rows:
            missing_rows = required_rows - current_rows
            zero_padding = pd.DataFrame(
                0, 
                index=range(missing_rows), 
                columns=df1.columns
            )
            padded_df = pd.concat([df1, zero_padding], ignore_index=True)
        elif current_rows > required_rows:
            padded_df = df1.iloc[:required_rows, :]
        else:
            padded_df = df1
        flattened_array = padded_df.to_numpy().flatten()
        reshaped_array = flattened_array.reshape(-1, 1)
        flattened_array2 = df2.to_numpy().flatten()
        flattened_array3 = df3.to_numpy().flatten()
        flattened_array4 = df4.to_numpy().flatten()
        stacked_array = np.concatenate([ reshaped_array.reshape(1, -1), 
                                        flattened_array2.reshape(1, -1),
                                        flattened_array3.reshape(1, -1),
                                        flattened_array4.reshape(1, -1)], axis=1)
        final_array = stacked_array.reshape(1, -1)
        x_train[counter] = final_array
    i=0 
    counter=0 
    x_train=np.zeros((24,126)) 
    team1_players=data['team1_players'].apply(ast.literal_eval)[0]
    team2_players=data['team2_players'].apply(ast.literal_eval)[0] 
    date=data['date'][0] 
    venue=data['venue'][0]  
    for player in team1_players:  
        a=get_player_venue_stats(player_stats_with_date_venue,player,venue,date,3000) 
        b=get_player_matchwise_stats(all_matches_player,player,date,180) 
        c=get_player_vs_player_stats_ordered(player_vs_player_stats,player,team2_players,date,800) 
        d=get_weather_data(venues_with_dates_with_locations_with_weather,date,venue) 
        stack_and_pad_dataframes(c,a,b,d,x_train,counter) 
        counter=counter+1
    for player in team2_players:  
        a=get_player_venue_stats(player_stats_with_date_venue,player,venue,date,3000) 
        b=get_player_matchwise_stats(all_matches_player,player,date,180) 
        c=get_player_vs_player_stats_ordered(player_vs_player_stats,player,team1_players,date,800) 
        d=get_weather_data(venues_with_dates_with_locations_with_weather,date,venue) 
        stack_and_pad_dataframes(c,a,b,d,x_train,counter)
        counter=counter+1   
    x=np.delete(x_train, [11, 23], axis=0)
    x = x.reshape(x.shape[0], -1)
    dtest = xgb.DMatrix(x)
    y = model.predict(dtest)  
    return y,x
    
    
    

In [20]:
import dill

with open("proceed.pkl", "wb") as f:
    dill.dump(proceed, f) 